# FaceWord Notebook 
### *- Laurits Lyngbæk*
This is a notebook for working through the EEG data from the face-word experiment

In [3]:
# Installing packages in the notebook
import os
import sys
!conda install --yes --prefix {sys.prefix} mne

Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.



PackagesNotFoundError: The following packages are not available from current channels:

  - mne

Current channels:

  - https://repo.anaconda.com/pkgs/main/win-64
  - https://repo.anaconda.com/pkgs/main/noarch
  - https://repo.anaconda.com/pkgs/r/win-64
  - https://repo.anaconda.com/pkgs/r/noarch
  - https://repo.anaconda.com/pkgs/msys2/win-64
  - https://repo.anaconda.com/pkgs/msys2/noarch

To search for alternate channels that may provide the conda package you're
looking for, navigate to

    https://anaconda.org

and use the search bar at the top of the page.




In [4]:
# Loading packages:
install
import mne
import numpy as np
import pandas as pd
import os
import matplotlib
 
%matplotlib inline

ModuleNotFoundError: No module named 'mne'

# Fixing data:
There is a problem with the timestamps, we export a csv file to fix the problems in R (im more adept in R)

In [ ]:
# Reading files with a hard coded data location
raw = mne.io.read_raw_brainvision("./Data/own_group8.vhdr", eog= ("EOG1", "EOG2"))
raw.load_data()

In [ ]:
# Save the timestamps as a csv file for manipulation in R
timestamps = mne.events_from_annotations(raw)[0]
np.savetxt("timestamps.csv", timestamps, delimiter=",")

# Import the manipulated timestamps from R
# Read the CSV file into a pandas DataFrame
df = pd.read_csv('Cleaned_Timestamps.csv')

# Convert the DataFrame to a numpy array
data = df.to_numpy()

# Save the manipulated timestamps to mne.events_from_annotations(raw)[0]
print(raw.annotations[0])



In [ ]:

# Selecting only EEG and stimulus data & plotting raw data
raw.plot(n_channels=32, duration=1, start = 300.5);

In [ ]:

#Specify the channel locations using standard_montage

montage = mne.channels.make_standard_montage('standard_1020')
raw.set_montage(montage, verbose=False)




In [ ]:
# Try to find bad channels using ICA:
raw.plot(n_channels=30, duration=60, start = 300, scalings=dict(eeg=50e-6));
#ICA
import sklearn # scikit-learn is needed to run ICA (Independent Component Analysis)
raw.set_eeg_reference(ref_channels='average', ch_type='eeg')
raw_fit = raw.copy().filter(1, None)



In [ ]:
ica = mne.preprocessing.ICA(n_components=0.95, random_state=7, max_iter=800)
ica.fit(raw_fit)
ica.plot_components();

In [ ]:
ica.plot_sources(raw_fit);

In [ ]:
ica.exclude = [0,1,2] # setting the exclude argument to the noise components
ica.plot_properties(raw, picks=ica.exclude); # plotting the components that will be excluded (sanity check, did we index the right ones?)

ica.apply(raw)

In [ ]:
#No obvious bad channels found

#lowpass filter
raw.filter(None, 40)
#Highpass filter
raw.filter(0.1, None)

In [ ]:
# it seems the intense amplitude and frequenzy of the Fz makes the data un-interprable
#raw.info['bads'] = ['Fz']
#raw.pick_types(meg=False, eeg=True, stim=True, exclude='bads')


#### COMMENTED OUT as this was not necessarry/warranted after ICA

In [ ]:
#Plotting after filtering:

raw.plot(n_channels=30, duration=20, start=300, scalings=dict(eeg=50e-6));

In [ ]:
event_id = {
    'Changed/Familliar/0' : 221,
    'Changed/Familliar/90' : 222,
    'Changed/Familliar/180' : 223,
    'Changed/Unfamilliar/0' : 211,
    'Changed/Unfamilliar/90' : 212,
    'Changed/Unfamilliar/180' : 213,
    'Unchanged/Familliar/0' : 121,
    'Unchanged/Familliar/90' : 122,
    'Unchanged/Familliar/180' : 123,
    'Unchanged/Unfamilliar/0' : 111,
    'Unchanged/Unfamilliar/90' : 112,
    'Unchanged/Unfamilliar/180' : 113,
    'Response_Changed' : 1,
    'Response_Unchanged' : 2,

}

In [ ]:
# Plot events :))
events = mne.events_from_annotations(raw)[0]
mne.viz.plot_events(events, sfreq=250, first_samp=raw.first_samp, event_id=event_id);



In [ ]:
# Define time window for epochs:
tmin, tmax = -0.1, 0.7
#create the rejection dictionary (how big the difference between a low and high can be in a (unrejected) epoch
reject = {'eeg': 150e-6} 
raw.plot(n_channels=30, duration=1, scalings=dict(eeg=20e-6));
epochs = mne.Epochs(raw, events, event_id, tmin, tmax, reject=reject, preload=True)
epochs




In [ ]:
# Downsample the data:
epochs_resampled = epochs.resample(250)

# ERP analysis
The data has been preprocessed and is ready to be analyzed-

In [ ]:
# Group the epochs by modality (i.e. visual and auditory)
word_pos = epochs_resampled['wPos']
word_neg = epochs_resampled['wNeg']
word_neu =  epochs_resampled['wNeu']
print(word_pos)


# and create evoked responses (averaged across trials)
evoked_word_pos = word_pos.average()
evoked_word_neg = word_neg.average()
evoked_word_neu = word_neu.average()
print(epoch_word_pos)

word_pos.plot_sensors(ch_type='eeg', show_names=True);

In [ ]:
mne.viz.plot_compare_evokeds(dict(positive=epoch_word_pos, negative=epoch_word_neg),
                             legend='upper left', show_sensors='upper right')

In [ ]:
pos_word_plot = word_pos.plot_image(picks='O1');
pos_word_plot = word_neg.plot_image(picks='O1');

In [ ]:
pos_word_plot[0]